# air 관련 데이터 크롤링

In [1]:
import json
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from ast import literal_eval

## 데이터를 가져올 url

In [2]:
url = 'https://api.openweathermap.org/data/2.5/air_pollution/forecast?lat=위도&lon=경도&appid=APIKEY'
# 북위 37.5642135° 동경 127.0016985°
url = url.replace('위도','37.5642135')
url = url.replace('경도','127.0016985')
url = url.replace('APIKEY','여기다가 발급받은 API KEY로 대체')

## 데이터 가져오기

In [3]:
response = requests.get(url)
data = response.json()

In [4]:
#data

## json 형식으로 데이터 저장

In [5]:
with open("air.json", "w") as json_file:
    json.dump(data, json_file)

## json파일 가져와서 데이터 가공

In [6]:
import pandas as pd
from datetime import datetime
from pytz import timezone
import time
load_data = json.load(open("air.json"))
KST = timezone('Asia/Seoul')
today = datetime.now(timezone('Asia/Seoul'))
today.day

17

In [7]:
# for i in range(0,len(load_data['list'])):
#   df = df.append(load_data['list'][i]['components'],ignore_index=True)
#   d_time = (datetime.strptime(time.strftime('%Y-%m-%d %H',time.gmtime(load_data['list'][i]['dt'])),"%Y-%m-%d %H")).astimezone(KST)
#   df['date'][i] = datetime.strftime(d_time,'%Y-%m-%d %H')

### pandas 데이터프레임에 저장

In [8]:
df = pd.DataFrame(columns = ['date'])
for i in range(0,len(load_data['list'])):
  d_time = (datetime.strptime(time.strftime('%Y-%m-%d %H',time.gmtime(load_data['list'][i]['dt'])),"%Y-%m-%d %H")).astimezone(KST) #한국시간으로 변경
  if (d_time.month==today.month) and (d_time.day < today.day):
     continue
  df = df.append(load_data['list'][i]['components'],ignore_index=True)
  df['date'].fillna(value = datetime.strftime(d_time,'%Y-%m-%d %H'),inplace=True)
df

,date,co,nh3,no,no2,o3,pm10,pm2_5,so2
0,2021-07-17 00,373.84,0.78,11.74,39.07,31.11,26.22,23.21,12.40
1,2021-07-17 01,263.69,0.45,3.55,25.02,39.70,13.41,11.83,8.82
2,2021-07-17 02,223.64,0.48,2.52,17.31,43.63,8.31,7.30,7.15
3,2021-07-17 03,210.29,0.59,2.10,14.91,52.21,6.56,5.74,6.56
4,2021-07-17 04,200.27,0.71,1.23,15.25,50.07,4.44,3.78,5.96
...,...,...,...,...,...,...,...,...,...
68,2021-07-19 20,540.73,1.03,52.75,25.02,0.00,41.78,39.35,7.57
69,2021-07-19 21,540.73,1.03,52.75,25.02,0.00,41.78,39.35,7.57
70,2021-07-19 22,540.73,1.03,52.75,25.02,0.00,41.78,39.35,7.57
71,2021-07-19 23,540.73,1.03,52.75,25.02,0.00,41.78,39.35,7.57


### 필요한 열만 추출

In [9]:
result = df[['date','co','pm10','pm2_5']]
result

,date,co,pm10,pm2_5
0,2021-07-17 00,373.84,26.22,23.21
1,2021-07-17 01,263.69,13.41,11.83
2,2021-07-17 02,223.64,8.31,7.30
3,2021-07-17 03,210.29,6.56,5.74
4,2021-07-17 04,200.27,4.44,3.78
...,...,...,...,...
68,2021-07-19 20,540.73,41.78,39.35
69,2021-07-19 21,540.73,41.78,39.35
70,2021-07-19 22,540.73,41.78,39.35
71,2021-07-19 23,540.73,41.78,39.35


## 결과 저장

In [10]:
result.to_csv("./air_{}.csv".format(datetime.strftime(today,"%y%m%d")),index=False,encoding='cp949')